In [1]:
from misc import *

%load_ext autoreload
%autoreload 2
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

# First we should load the .mat file
data_dir = "C:/Users/Mircea/Google Drive/Data/nn-dipole-fitting/"
path_list = [each.replace("\\","/") for each in glob(data_dir + "Dataset SNR*.mat")]

# check if CUDA is available
use_cuda = torch.cuda.is_available()
if not use_cuda:
    print('CUDA is not available.  Training on CPU ...')
    device = "cpu"
else:
    print('CUDA is available!  Training on GPU ...')
    device = torch.device("cuda:0")
    print("Using",torch.cuda.get_device_name(device))

CUDA is available!  Training on GPU ...
Using GeForce RTX 2070


In [2]:
# Read the data if not already in memory
if "data_train" not in globals():
    dataset = dataset_from_mat(path_list[-1])

min_location = torch.from_numpy(dataset.data_train["min_location"].T).to(device)
max_location = torch.from_numpy(dataset.data_train["max_location"].T).to(device)
min_moment = torch.from_numpy(dataset.data_train["min_moment"].T).to(device)
max_moment = torch.from_numpy(dataset.data_train["max_moment"].T).to(device)

In [3]:
# Define parameters
batch_size = 50000
n_epochs = 75

In [4]:
model = dipfit(dataset.n_chan,0)
model = model.to(device)
weight_localization = 0.5
optimizer = optim.Adam(model.parameters(),3e-4) 

In [5]:
model, train_loss, valid_loss = train(model,dataset,n_epochs,batch_size,device,optimizer,weight_localization,"best_model.pt",
     min_location,max_location,min_moment,max_moment)

Epoch 1 done in 3.00 seconds. 	Training Loss: 0.035621805 	Validation Loss: 0.016545072
Epoch 2 done in 2.61 seconds. 	Training Loss: 0.011539835 	Validation Loss: 0.009305499
Epoch 3 done in 2.61 seconds. 	Training Loss: 0.008404497 	Validation Loss: 0.007796114
Epoch 4 done in 2.62 seconds. 	Training Loss: 0.007392371 	Validation Loss: 0.007084405
Epoch 5 done in 2.61 seconds. 	Training Loss: 0.006827597 	Validation Loss: 0.006646983
Epoch 6 done in 2.62 seconds. 	Training Loss: 0.006461239 	Validation Loss: 0.006351945
Epoch 7 done in 2.63 seconds. 	Training Loss: 0.006215000 	Validation Loss: 0.006157367
Epoch 8 done in 2.64 seconds. 	Training Loss: 0.006036482 	Validation Loss: 0.006009422
Epoch 9 done in 2.63 seconds. 	Training Loss: 0.005917590 	Validation Loss: 0.005889713
Epoch 10 done in 2.62 seconds. 	Training Loss: 0.005820314 	Validation Loss: 0.005822338
Epoch 11 done in 2.63 seconds. 	Training Loss: 0.005736969 	Validation Loss: 0.005737299
Epoch 12 done in 2.63 seconds.

In [6]:
# Calculate test loss for the network
x = torch.from_numpy(dataset.data_test["fields"]).to(device)
y = torch.from_numpy(dataset.data_test["dipoles"]).to(device)
output = model(x)
test_loss_nn = weighted_mse(normalize_dipole(output,dataset.data_test,device,min_location,max_location,min_moment,max_moment), 
                 normalize_dipole(y,dataset.data_test,device,min_location,max_location,min_moment,max_moment), weight_localization).item()

# Test loss for Levenberg-Marquadt
x = torch.from_numpy(dataset.data_test["dipoles_estimated"]).to(device)
y = torch.from_numpy(dataset.data_test["dipoles"]).to(device).squeeze()
test_loss_lm = weighted_mse(normalize_dipole(x,dataset.data_test,device,min_location,max_location,min_moment,max_moment), 
                 normalize_dipole(y,dataset.data_test,device,min_location,max_location,min_moment,max_moment), weight_localization)
print(f"Test loss: \nNN: {test_loss_nn:.6f}\nLM: {test_loss_lm:.6f}")

# Localization errors
loc_error_nn = np.mean(get_localization_error(output.to("cpu").detach().numpy(),dataset.data_test["dipoles"]))
loc_error_lm = np.mean(get_localization_error(dataset.data_test["dipoles_estimated"],dataset.data_test["dipoles"]))
print(f"\nLocalization error: \nNN: {loc_error_nn:.3f} cm\nLM: {loc_error_lm:.3f} cm")

Test loss: 
NN: 0.004625
LM: 0.011570

Localization error: 
NN: 1.115 cm
LM: 1.071 cm
